In [18]:
from aeon.datasets import (
    load_anomaly_detection,
    load_classification,
    load_forecasting,
    load_regression,
)
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, RidgeClassifierCV, RidgeClassifier
import pywt
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField
from pyts.image import RecurrencePlot
from aeon.transformations.detrend import ConditionalDeseasonalizer

# from all_functions import *

def znorm(x):
  x_znorm = (x - np.mean(x)) / np.std(x)
  return x_znorm


def normalize_series_znorm(X):
    rep = []
    # X_train_series = X.flatten()

    # transform = ConditionalDeseasonalizer(sp=12)
    # transform.fit(X_train_series)
    # X_train_deseasonal = transform.transform(X_train_series)
    # X_train_flat_reshaped = X_train_deseasonal.reshape(X.shape)

    X_train_flat_reshaped = X
    for x in X_train_flat_reshaped:
      # x_normalized = znorm(x)
      x_normalized = x
      rep.append(transform_series2(x_normalized, "DWT", "bior2.2", 2))
    new = np.array(rep)
    return new.reshape(new.shape[0], -1)

def flatten_series(X):
    return X.reshape(X.shape[0], -1)
  
def flatten_labels(y):
    return np.ravel(y)

def transform_series2(series, representation, wavelet, level):
  # series = np.array(znorm(series))
  if representation == "CWT":
    coeffs, freqs = pywt.cwt(series, scales=np.arange(1, len(series) + 1), wavelet=wavelet) # morl
    im_final = coeffs
  elif representation == "DWT":
    coeffs = pywt.wavedec(series, wavelet=wavelet, level=level)
    im_final = np.concatenate(coeffs, axis=0) 
  elif representation == "SWT":
    coeffs_swt = pywt.swt(series, wavelet, level=level)
    im_final = np.concatenate([coeff[0] for coeff in coeffs_swt] + [coeff[1] for coeff in coeffs_swt], axis=0)

  elif representation == "WPT":
    wp = pywt.WaveletPacket(data=series, wavelet='db1', mode='symmetric')

    # Extrair os coeficientes em diferentes níveis
    nodes = wp.get_level(4, order='freq')  # Pegando o 4º nível de decomposição
    coeffs_wpt = np.array([n.data for n in nodes])

    # Concatenar os coeficientes para visualização
    im_final = np.concatenate(coeffs_wpt, axis=0)
  elif representation == "STFT":
    from scipy.signal import stft
    f, t, Zxx = stft(series,window='hann', nperseg=64)

    # Obter a magnitude dos coeficientes
    coeffs_stft = np.abs(Zxx)

    # Concatenar os coeficientes para criar im_final
    im_final = coeffs_stft
    

  elif representation == "MTF":
    series = series.reshape(1, len(series))
    mtf = MarkovTransitionField(strategy='normal') #n_bins=4, strategy='uniform'
    X_mtf = mtf.fit_transform(series)
    im_final = X_mtf[0]
  elif representation == "GADF":
    series = series.reshape(1, len(series))
    gaf = GramianAngularField(method='difference')
    X_gaf = gaf.fit_transform(series)
    im_final = X_gaf[0]
  elif representation == "GASF":
    series = series.reshape(1, len(series))
    gaf = GramianAngularField(method='summation')
    X_gaf = gaf.fit_transform(series)
    im_final = X_gaf[0]
  elif representation == "RP":
    series = series.reshape(1, len(series))
    rp = RecurrencePlot(threshold='distance')
    X_rp = rp.fit_transform(series)
    im_final = X_rp[0]
  elif representation == "FIRTS":
    series = series.reshape(1, len(series))
    mtf = MarkovTransitionField(n_bins=4, strategy='uniform')
    X_mtf = mtf.fit_transform(series)
    gaf = GramianAngularField(method='difference')
    X_gaf = gaf.fit_transform(series)
    rp = RecurrencePlot(threshold='distance')
    X_rp = rp.fit_transform(series)
    im_final = (X_mtf[0] + X_gaf[0] + X_rp[0]) # FIRTS é fusão entre MTF, GADF e RP (vejam o artigo que passei para vocês)
  return im_final


In [23]:
X, y, meta = load_classification("Crop", return_metadata=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_flattened = flatten_series(X_train)
X_test_flattened = flatten_series(X_test)

X_train_normalized = normalize_series_znorm(X_train_flattened)
X_test_normalized = normalize_series_znorm(X_test_flattened)

In [ ]:
y_train_flattened = flatten_labels(y_train)
y_test_flattened = flatten_labels(y_test)
from catboost import CatBoostClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


# clf = RidgeClassifierCV()
clf = RandomForestClassifier(n_estimators=500, random_state=42)
# clf = KNeighborsClassifier(n_neighbors=5)



clf.fit(X_train_normalized, y_train)

y_pred = clf.predict(X_test_normalized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy * 100:.2f}%")